In [94]:
# Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime
import seaborn
import warnings

In [95]:
# Read the file
accidents = pd.read_csv("D:\\capstone project\\NCDB_1999_to_2017.csv")

D:\anaconda\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1,2,5,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [96]:
# See the dataset rows
accidents.shape[0]

6772563

In [97]:
# see the dataset columns
accidents.shape[1]

23

In [98]:
# Filtering data from year 2012
accidents_new = accidents[accidents['C_YEAR']>=2012]

In [99]:
accidents_new.head()

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,...,V_TYPE,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER,C_CASE
4923314,2012,01,1,10,2,01,02,03,1,3,...,01,UUUU,01,M,75,11,2,NN,1,1855036
4923315,2012,01,1,12,2,01,04,UU,1,5,...,01,UUUU,01,F,21,11,2,02,1,1855045
4923316,2012,01,1,00,2,01,03,UU,7,3,...,01,UUUU,01,F,34,11,2,02,1,1855084
4923317,2012,01,1,17,2,02,21,UU,1,1,...,01,UUUU,01,F,50,11,2,02,1,1855190
4923318,2012,01,1,17,2,02,21,UU,1,1,...,01,UUUU,01,M,63,11,1,NN,1,1855190


In [100]:
# The dataset rows after filtering from year 2012 to 2017
accidents_new.shape[0]

1849249

In [101]:
accidents_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1849249 entries, 4923314 to 6772562
Data columns (total 23 columns):
C_YEAR    int64
C_MNTH    object
C_WDAY    object
C_HOUR    object
C_SEV     int64
C_VEHS    object
C_CONF    object
C_RCFG    object
C_WTHR    object
C_RSUR    object
C_RALN    object
C_TRAF    object
V_ID      object
V_TYPE    object
V_YEAR    object
P_ID      object
P_SEX     object
P_AGE     object
P_PSN     object
P_ISEV    object
P_SAFE    object
P_USER    object
C_CASE    int64
dtypes: int64(3), object(20)
memory usage: 338.6+ MB


In [102]:
# missing data in dataset
accidents_new.isnull().sum()

C_YEAR    0
C_MNTH    0
C_WDAY    0
C_HOUR    0
C_SEV     0
C_VEHS    0
C_CONF    0
C_RCFG    0
C_WTHR    0
C_RSUR    0
C_RALN    0
C_TRAF    0
V_ID      0
V_TYPE    0
V_YEAR    0
P_ID      0
P_SEX     0
P_AGE     0
P_PSN     0
P_ISEV    0
P_SAFE    0
P_USER    0
C_CASE    0
dtype: int64

In [103]:
data_columns = ['C_YEAR','C_MNTH','C_WDAY']

In [104]:
accidents_new = (accidents_new.drop(data_columns, axis=1).join(accidents_new[data_columns].apply(pd.to_numeric, errors='coerce')))

In [105]:
accidents_new.tail()

,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,C_TRAF,V_ID,...,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER,C_CASE,C_YEAR,C_MNTH,C_WDAY
6772558,UU,2,UU,UU,01,U,U,U,UU,11,...,F,20,11,1,NN,1,2455828,2017,NaN,NaN
6772559,UU,2,UU,UU,01,U,U,U,UU,12,...,F,47,11,1,NN,1,2455828,2017,NaN,NaN
6772560,UU,2,UU,UU,01,U,U,U,UU,13,...,M,24,11,1,NN,1,2455828,2017,NaN,NaN
6772561,23,2,01,03,01,1,1,2,18,01,...,M,45,96,2,09,U,2570233,2017,NaN,NaN
6772562,23,2,01,03,01,1,1,2,18,01,...,F,45,96,2,09,U,2570233,2017,NaN,NaN


In [106]:
accidents_new_1 = accidents_new[accidents_new[data_columns].notnull().all(axis=1)]

In [107]:
accidents_new_1.shape[0]

1849089

In [108]:
accidents_new_1.tail()

,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,C_TRAF,V_ID,...,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER,C_CASE,C_YEAR,C_MNTH,C_WDAY
6772536,19,2,01,QQ,02,2,2,2,03,99,...,U,UU,99,2,NN,3,2569967,2017,12.0,7.0
6772537,22,2,01,04,01,4,5,1,18,01,...,M,39,11,U,02,1,2570203,2017,12.0,7.0
6772538,22,2,01,04,01,4,5,1,18,01,...,M,61,13,U,02,2,2570203,2017,12.0,7.0
6772539,22,2,01,04,01,4,5,1,18,01,...,M,40,21,U,02,2,2570203,2017,12.0,7.0
6772540,22,2,01,04,01,4,5,1,18,01,...,M,38,23,2,02,2,2570203,2017,12.0,7.0


In [109]:
accidents_8909new_1 = accidents_new_1.astype({"C_YEAR":int,"C_MNTH":int,"C_WDAY":int})

In [110]:
accidents_new_1.tail()

,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,C_TRAF,V_ID,...,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER,C_CASE,C_YEAR,C_MNTH,C_WDAY
6772536,19,2,01,QQ,02,2,2,2,03,99,...,U,UU,99,2,NN,3,2569967,2017,12,7
6772537,22,2,01,04,01,4,5,1,18,01,...,M,39,11,U,02,1,2570203,2017,12,7
6772538,22,2,01,04,01,4,5,1,18,01,...,M,61,13,U,02,2,2570203,2017,12,7
6772539,22,2,01,04,01,4,5,1,18,01,...,M,40,21,U,02,2,2570203,2017,12,7
6772540,22,2,01,04,01,4,5,1,18,01,...,M,38,23,2,02,2,2570203,2017,12,7


In [111]:
accidents_new_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1849089 entries, 4923314 to 6772540
Data columns (total 23 columns):
C_HOUR    object
C_SEV     int64
C_VEHS    object
C_CONF    object
C_RCFG    object
C_WTHR    object
C_RSUR    object
C_RALN    object
C_TRAF    object
V_ID      object
V_TYPE    object
V_YEAR    object
P_ID      object
P_SEX     object
P_AGE     object
P_PSN     object
P_ISEV    object
P_SAFE    object
P_USER    object
C_CASE    int64
C_YEAR    int32
C_MNTH    int32
C_WDAY    int32
dtypes: int32(3), int64(2), object(18)
memory usage: 317.4+ MB


In [114]:
accidents_new_1['Date'] = pd.to_datetime(dict(year=accidents_new_1.C_YEAR, month=accidents_new_1.C_MNTH, day=accidents_new_1.C_WDAY))

In [115]:
accidents_new_1.head()

,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,C_TRAF,V_ID,...,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER,C_CASE,C_YEAR,C_MNTH,C_WDAY,Date
4923314,10,2,01,02,03,1,3,2,18,1,...,75,11,2,NN,1,1855036,2012,1,1,2012-01-01
4923315,12,2,01,04,UU,1,5,1,UU,1,...,21,11,2,02,1,1855045,2012,1,1,2012-01-01
4923316,00,2,01,03,UU,7,3,1,UU,1,...,34,11,2,02,1,1855084,2012,1,1,2012-01-01
4923317,17,2,02,21,UU,1,1,1,18,1,...,50,11,2,02,1,1855190,2012,1,1,2012-01-01
4923318,17,2,02,21,UU,1,1,1,18,2,...,63,11,1,NN,1,1855190,2012,1,1,2012-01-01


In [116]:
accidents_new_1.set_index('Date', inplace=True)

In [117]:
accidents_new_1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1849089 entries, 2012-01-01 to 2017-12-07
Data columns (total 23 columns):
C_HOUR    object
C_SEV     int64
C_VEHS    object
C_CONF    object
C_RCFG    object
C_WTHR    object
C_RSUR    object
C_RALN    object
C_TRAF    object
V_ID      object
V_TYPE    object
V_YEAR    object
P_ID      object
P_SEX     object
P_AGE     object
P_PSN     object
P_ISEV    object
P_SAFE    object
P_USER    object
C_CASE    int64
C_YEAR    int32
C_MNTH    int32
C_WDAY    int32
dtypes: int32(3), int64(2), object(18)
memory usage: 317.4+ MB


In [119]:
accidents_new_1 = accidents_new_1[['C_YEAR','C_MNTH','C_WDAY','C_HOUR','C_SEV','C_VEHS','C_CONF','C_RCFG','C_WTHR','C_RSUR','C_RALN','C_TRAF','V_ID','V_TYPE','V_YEAR','P_ID','P_SEX','P_AGE','P_PSN','P_ISEV','P_SAFE','P_USER','C_CASE']]

In [120]:
accidents_new_1.head()

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,...,V_TYPE,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER,C_CASE
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-01,2012,1,1,10,2,01,02,03,1,3,...,01,UUUU,01,M,75,11,2,NN,1,1855036
2012-01-01,2012,1,1,12,2,01,04,UU,1,5,...,01,UUUU,01,F,21,11,2,02,1,1855045
2012-01-01,2012,1,1,00,2,01,03,UU,7,3,...,01,UUUU,01,F,34,11,2,02,1,1855084
2012-01-01,2012,1,1,17,2,02,21,UU,1,1,...,01,UUUU,01,F,50,11,2,02,1,1855190
2012-01-01,2012,1,1,17,2,02,21,UU,1,1,...,01,UUUU,01,M,63,11,1,NN,1,1855190
